In [23]:
import pandas as pd
import json

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1600)

In [7]:
with open('jo1.json','r') as openfile:
    jo1 = json.load(openfile)
    
with open('jo2.json','r') as openfile:
    jo2 = json.load(openfile)
    
with open('jo3.json','r') as openfile:
    jo3 = json.load(openfile)
    
with open('mo1.json','r') as openfile:
    mo1 = json.load(openfile)
    
with open('mo2.json','r') as openfile:
    mo2 = json.load(openfile)
    
with open('mo3.json','r') as openfile:
    mo3 = json.load(openfile)

In [25]:
df_jo1 = pd.json_normalize(jo1, record_path=['results'])
df_jo2 = pd.json_normalize(jo2, record_path=['results'])
df_jo3 = pd.json_normalize(jo3, record_path=['results'])

df_mo1 = pd.json_normalize(mo1, record_path=['results'])
df_mo2 = pd.json_normalize(mo2, record_path=['results'])
df_mo3 = pd.json_normalize(mo3, record_path=['results'])

In [34]:
jo_frames = [df_jo1,df_jo2,df_jo3]
df_jo = pd.concat(jo_frames,axis=0)

In [40]:
mo_frames = [df_mo1,df_mo2,df_mo3]
df_mo = pd.concat(mo_frames,axis=0)

In [47]:
df_jo.columns

Index(['business_status', 'formatted_address', 'icon', 'icon_background_color',
       'icon_mask_base_uri', 'name', 'photos', 'place_id', 'price_level',
       'rating', 'reference', 'types', 'user_ratings_total',
       'geometry.location.lat', 'geometry.location.lng',
       'geometry.viewport.northeast.lat', 'geometry.viewport.northeast.lng',
       'geometry.viewport.southwest.lat', 'geometry.viewport.southwest.lng',
       'opening_hours.open_now', 'plus_code.compound_code',
       'plus_code.global_code'],
      dtype='object')

In [46]:
import folium
from folium.plugins import MarkerCluster

In [56]:
m = folium.Map(location=df_jo[["geometry.location.lat", "geometry.location.lng"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Jewel-Osco & Mariano\'s')

for i,r in df_jo.iterrows():
    location = (r["geometry.location.lat"], r["geometry.location.lng"])
    info = (r["name"], r["formatted_address"], r["rating"], r['price_level'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Name: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}{new_line}Rating: \
    {bold_start}{info[2]}{bold_end}{new_line}Price Level: {bold_start}{info[3]}'
    folium.Marker(location=location,icon=folium.Icon(color='lightblue'),
                      tooltip=text)\
    .add_to(m)
    folium.Circle(location=location,radius=4800).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

for i,r in df_mo.iterrows():
    location = (r["geometry.location.lat"], r["geometry.location.lng"])
    info = (r["name"], r["formatted_address"], r["rating"], r['price_level'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Name: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}{new_line}Rating: \
    {bold_start}{info[2]}{bold_end}{new_line}Price Level: {bold_start}{info[3]}'
    folium.Marker(location=location,icon=folium.Icon(color='darkpurple'),
                      tooltip=text)\
    .add_to(m)

m

In [57]:
m.save('index.html')

## Map URL snagger

In [58]:
Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

SyntaxError: invalid syntax (483186673.py, line 1)